# Cell finding demo

In [ ]:
import numpy as np

import seaborn as sns
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import matplotlib.ticker as plticker
import matplotlib.gridspec as gridspec

In [ ]:
OUTDIR = '../out/'
DATADIR = '../data/'

### Just some matplotlib and seaborn parameter tuning

In [ ]:
axistitlesize = 20
axisticksize = 17
axislabelsize = 26
axislegendsize = 23
axistextsize = 20
axiscbarfontsize = 15

# Set axtick dimensions
major_size = 6
major_width = 1.2
minor_size = 3
minor_width = 1
mpl.rcParams['xtick.major.size'] = major_size
mpl.rcParams['xtick.major.width'] = major_width
mpl.rcParams['xtick.minor.size'] = minor_size
mpl.rcParams['xtick.minor.width'] = minor_width
mpl.rcParams['ytick.major.size'] = major_size
mpl.rcParams['ytick.major.width'] = major_width
mpl.rcParams['ytick.minor.size'] = minor_size
mpl.rcParams['ytick.minor.width'] = minor_width

mpl.rcParams.update({'figure.autolayout': False})

# Seaborn style settings
sns.set_style({'axes.axisbelow': True,
               'axes.edgecolor': '.8',
               'axes.facecolor': 'white',
               'axes.grid': True,
               'axes.labelcolor': '.15',
               'axes.spines.bottom': True,
               'axes.spines.left': True,
               'axes.spines.right': True,
               'axes.spines.top': True,
               'figure.facecolor': 'white',
               'font.family': ['sans-serif'],
               'font.sans-serif': ['Arial',
                'DejaVu Sans',
                'Liberation Sans',
                'Bitstream Vera Sans',
                'sans-serif'],
               'grid.color': '.8',
               'grid.linestyle': '--',
               'image.cmap': 'rocket',
               'lines.solid_capstyle': 'round',
               'patch.edgecolor': 'w',
               'patch.force_edgecolor': True,
               'text.color': '.15',
               'xtick.bottom': True,
               'xtick.color': '.15',
               'xtick.direction': 'in',
               'xtick.top': True,
               'ytick.color': '.15',
               'ytick.direction': 'in',
               'ytick.left': True,
               'ytick.right': True})

# Colorpalettes, colormaps, etc.
sns.set_palette(palette='rocket')
rocket_cmap = sns.color_palette('rocket', as_cmap=True)

In [ ]:
def set_grid(nrows, ncols, stepx, stepy):
    grid = np.zeros((nrows*ncols, 2))
    # Calculate coordinates of grid points
    for i in range(nrows):
        for j in range(ncols):
            grid[i * ncols + j] = np.array((j*stepx, i*stepy))
            
    return grid

In [ ]:
nrows = 10
ncols = 10
res = 30

## 1. Finding main grid cells

In [ ]:
def set_cell_corners(nrows, ncols):
    crows = nrows - 1
    ccols = ncols - 1
    cell_corners = np.zeros((crows*ccols,4), dtype=int)
    
    #Iterate over all points, except the last row and the last column.
    #The iteration is basically over the possible upper left corners for a cell.
    for i in range(crows):
        for j in range(ccols):
            idx = i * ccols + j
            c = i * ncols + j

            cell_corners[idx] = np.array((c, c+1, c+ncols, c+ncols+1))
            
    return cell_corners

In [ ]:
def get_current_cell(p, nrows, ncols,
                     cell_corners):
    ix = (int)(p[0])
    iy = (int)(p[1])
    # Correct for points on borders
    if(ix == ncols-1): ix = ncols-2
    if(iy == nrows-1): iy = nrows-2

    return cell_corners[iy * (ncols-1) + ix]

In [ ]:
p = np.array((0.314*ncols, 0.35*nrows))

main_grid = set_grid(nrows, ncols, 1, 1)
cell_corners = set_cell_corners(nrows, ncols)
cell = get_current_cell(p, nrows, ncols,
                        cell_corners)

In [ ]:
fig, axes = plt.subplots(figsize=(10,10))
axes.set_aspect('equal')
axes.axis('off')

axes.scatter(main_grid[:,0], main_grid[:,1],
             color='grey', s=10**2, zorder=2)

for c in cell:
    axes.scatter(main_grid[c,0], main_grid[c,1],
                 color='tab:green', s=10**2, zorder=3)

for i in range(ncols):
    axes.plot([i, i], [0, nrows-1],
              color='grey', ls='--', lw=2, zorder=1)
for i in range(nrows):
    axes.plot([0, ncols-1], [i, i],
              color='grey', ls='--', lw=2, zorder=1)
    
axes.scatter(p[0], p[1],
             color='tab:red', marker='x', lw=4, s=15**2, zorder=4)
    
plt.show()

## 2. Finding subgrid cells

In [ ]:
def set_sub_cell_corners(nrows, ncols, res):
    crows = res-1
    ccols = res-1
    sub_cell_corners = np.zeros((crows*ccols,4), dtype=int)
    
    # Iterate over all points, except the last row and the last column.
    # The iteration is basically over the possible upper left corners for a cell.
    for i in range(crows):
        for j in range(ccols):
            idx = i * ccols + j
            c = i * (ccols+1) + j

            sub_cell_corners[idx] = np.array((c, c+1, c+ccols+1, c+ccols+2))
            
    return sub_cell_corners

In [ ]:
def get_current_sub_cell(p, nrows, ncols, res,
                         sub_cell_corners):
    ix = (int)(p[0] / ((ncols-1)/(res-1)))
    iy = (int)(p[1] / ((nrows-1)/(res-1)))

    snrows = res
    sncols = res
    # Correct for points on borders
    if(ix == sncols-1): ix = sncols-2
    if(iy == snrows-1): iy = snrows-2

    return sub_cell_corners[iy * (sncols-1) + ix]

In [ ]:
p = np.array((0.314*ncols, 0.35*nrows))

sub_grid = set_grid(res, res, (ncols-1)/(res-1), (nrows-1)/(res-1))
sub_cell_corners = set_sub_cell_corners(nrows, ncols, res)
sub_cell = get_current_sub_cell(p, nrows, ncols, res,
                                sub_cell_corners)

In [ ]:
fig, axes = plt.subplots(figsize=(10,10))
axes.set_aspect('equal')
axes.axis('off')

axes.scatter(main_grid[:,0], main_grid[:,1],
             color='grey', s=10**2, zorder=2)
axes.scatter(sub_grid[:,0], sub_grid[:,1],
             color='grey', s=3**2, zorder=2)

#for c in sub_cell:
for c in sub_cell:
    axes.scatter(sub_grid[c,0], sub_grid[c,1],
                 color='tab:green', s=8**2, zorder=3)

for i in range(ncols):
    axes.plot([i, i], [0, nrows-1],
              color='grey', ls='--', lw=2, zorder=1)
for i in range(nrows):
    axes.plot([0, ncols-1], [i, i],
              color='grey', ls='--', lw=2, zorder=1)
    
axes.scatter(p[0], p[1],
             color='tab:red', marker='x', lw=2, s=10**2, zorder=4)
    
plt.show()